# Setup
### Installing widgets & sounderpy, allowing widgets in Colab:

In [9]:
!pip install sounderpy ipywidgets > /dev/null 2>&1
from google.colab import output
output.enable_custom_widget_manager()

# Model Soundings Widget

In [20]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import sounderpy as spy

site_dict = {
    'DIA':'kden',
    'Denver - Stapleton':'DEN',
    'Centennial':'kapa',
    'Colorado Springs':'kcos',
    'Pueblo':'kpub',
    'Greeley':'KGXY',
    'Fort Morgan':'kfmm',
    'Limon':'klic',
    'Kit Carson':'kc1',
    'Sterling':'kstk',
    'Akron':'kako',
    'Wray':'K2V5',
    'Cheyenne':'KCYS',
    'Sydney':'sny',
    'La Junta':'LHX',
    'Lamar':'LAA',
    'Longmont':'tbl',
    'Broomfield':'KBJC',
    'Copper Mountain':'KCCU',
    'Kremmling':'K20V'
}

model_dict = {
    'HRRR':'HRRR',
    'NAM3k':'namnest',
    'NAM12k':'nam',
    'GFS':'GFS',
    'RAP':'RAP',
    'Short-Range Ensemble (SREF)':'sref',
    'HiRes WRF':'hiresw',
}
# --- Dropdown Widgets ---
site_dropdown = widgets.Dropdown(
    options=list(site_dict.keys()),  # Add more sites if needed
    value='DIA',
    description='Site:',
)

model_dropdown = widgets.Dropdown(
    options=list(model_dict.keys()),
    value='HRRR',
    description='Model:',
)

forecast_dropdown = widgets.Dropdown(
    options=[0, 3, 6, 9, 12, 15, 18, 21],
    value=6,
    description='Forecast Hour (UTC):',
)

generate_button = widgets.Button(
    description='Generate Sounding',
    button_style='success',
)

output = widgets.Output()

# --- Event Handler ---
def on_generate_clicked(b):
    with output:
        clear_output(wait=True)
        try:
            site_id = site_dict[site_dropdown.value]
            model_name = model_dict[model_dropdown.value]
            forecast_hour = forecast_dropdown.value
            model_data = spy.get_bufkit_data(model_name, site_id, forecast_hour)
            spy.build_sounding(model_data, 'full', True, True)
        except Exception as e:
            print("Error:", e)

generate_button.on_click(on_generate_clicked)

# --- Display UI ---
ui = widgets.VBox([
    widgets.HBox([site_dropdown, model_dropdown, forecast_dropdown]),
    generate_button,
    output
])

display(ui)

# ACARS Soundings widget

In [18]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import sounderpy as spy
from datetime import datetime

today = datetime.utcnow()

year_widget = widgets.Text(value=str(today.year), description='Year:')
month_widget = widgets.Text(value=f"{today.month:02}", description='Month:')
day_widget = widgets.Text(value=f"{today.day:02}", description='Day:')
hour_widget = widgets.Dropdown(
    options=[f"{h:02}" for h in range(0, 24, 1)],
    value=f"{today.hour:02}",
    description='Hour:'
)

acars_profile_dropdown = widgets.Dropdown(
    options=[],
    description='Profile:',
)

load_profiles_button = widgets.Button(description='Load Profiles', button_style='info')
generate_button = widgets.Button(description='Generate Sounding', button_style='success')
output = widgets.Output()

acars_conn = None

def on_load_profiles_clicked(b):
    global acars_conn
    with output:
        clear_output(wait=True)
        y, m, d, h = year_widget.value, month_widget.value, day_widget.value, hour_widget.value
        try:
            acars_conn = spy.acars_data(y, m, d, h)
            profiles = acars_conn.list_profiles()
            if profiles:
                acars_profile_dropdown.options = profiles
            else:
                print("No profiles found for this time.")
        except Exception as e:
            print("Error loading ACARS data:", e)

def on_generate_clicked(b):
    with output:
        clear_output(wait=True)
        try:
            profile = acars_profile_dropdown.value
            clean_data = acars_conn.get_profile(profile)
            spy.build_sounding(clean_data)
        except Exception as e:
            print("Error generating sounding:", e)

load_profiles_button.on_click(on_load_profiles_clicked)
generate_button.on_click(on_generate_clicked)

ui = widgets.VBox([
    widgets.HBox([year_widget, month_widget, day_widget, hour_widget]),
    load_profiles_button,
    acars_profile_dropdown,
    generate_button,
    output
])

display(ui)